<a href="https://colab.research.google.com/github/mlKwon/crawling/blob/main/amazon_shopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,512 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcont

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os
import gc
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + user_agent)
options.add_argument('--start-maximized')
options.add_argument('incognito')
service = Service()

try:
    driver = webdriver.Chrome(service=service, options=options)
except Exception as e:
    print(e)


In [34]:
driver.quit()
service = Service()
driver = webdriver.Chrome(service=service, options=options)
url = "https://www.amazon.com/stores/page/BDB9CC0A-DA48-41E3-ACD8-B0255744997D?ingress=2&visitId=a0a7c6d0-3822-4620-98ac-077900f58183&ref_=ast_bln" # Amazon 넘버즈인 브랜드관
driver.get(url)
driver.implicitly_wait(1)

temp = driver.find_elements(By.CSS_SELECTOR, 'span.Navigation__linkText__LoQD4.Navigation__en-US__YUfTQ') # Category

temp = temp[2:-1]
temp_text = [tt.text for tt in temp] # Category name
temp_link = driver.find_elements(By.CSS_SELECTOR, 'li.Navigation__navItem__bakjf a')[2:-1]
temp_link = [link.get_attribute('href') for link in temp_link] # Category link

link_prd_len = [7,10,5,3,1,2,5] # number of products for each category : 23.12.09

review_all = []
global_i = 0
b_stopped = False

for i, link in enumerate(temp_link):
  if i<1: continue
  print('')
  print(temp_text[i])

  driver.get(link)

  if temp_text[i] not in ['CLEANSER',"SUNSCREEN",'TONE UP']:
    selector = 'div.ProductGridItem__imageContainer__Ts5N_.default a'
  else:
    # selector = 'a.ProductShowcase__title__SBCBw'
    selector = 'a.ProductShowcase__rating__Wi83c'

  product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)] # product link
  while len(product_link) < link_prd_len[i]:
    product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)]

  print(f'link 수:{len(product_link)}')

  for j, prd_link in enumerate(product_link):
    if (i==1) & (j<3): continue
    review_dict = {'prd_type':'0','prd_name':'a','title':'d','star':'c','review':'e','date':'b','country':'cc'}

    driver.get(prd_link)
    prd_name = driver.find_elements(By.CSS_SELECTOR, '#productTitle')
    prd_name = prd_name[len(prd_name)-1].text
    print(f'\n{j+1}=> {prd_name}')

    # go to review
    while True:
      try:
        if(b_stopped):
          driver.quit()
          service = Service()
          driver = webdriver.Chrome(service=service, options=options)
          driver.get(prd_link)
          driver.implicitly_wait(1)
        try:
          review_link = driver.find_elements(By.CSS_SELECTOR,'div.a-row.a-spacing-medium a')[1].get_attribute('href') # US review
          b_skip_title = True
        except:
          review_link = driver.find_elements(By.CSS_SELECTOR,'a[data-hook="see-all-reviews-link-foot"]')[0].get_attribute('href') # non-US review
          b_skip_title = False

        driver.get(review_link)

        rating_info_element = driver.find_element(By.CSS_SELECTOR, 'div[data-hook="cr-filter-info-review-rating-count"]')
        rating_info_text = rating_info_element.text.strip()
        # 정규표현식을 사용하여 숫자 추출
        match = re.search(r'(\d+) total ratings, (\d+) with reviews', rating_info_text)

        if match:
            total_ratings = match.group(1)
            with_reviews = match.group(2)

        print(f'number of reviews: {with_reviews}')

        n_review = int(with_reviews)
        if n_review % 10 > 0:
          n_rep = n_review//10 + 1
        else:
          n_rep = n_review//10

        print('pages: ', end='')
        for rep in range(n_rep):
          time.sleep(.5)

          if b_skip_title: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')[2:]]
          else: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')]
          review_date = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,"span[data-hook='review-date']")]
          review_country = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(1) for tt in review_date]
          review_date = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(2) for tt in review_date]
          review_star = [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='review-star-rating'] span.a-icon-alt")]
          if len(review_star) < len(review_date): # other language
            review_star = review_star + [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='cmps-review-star-rating'] span.a-icon-alt")]
          review_star = [re.search(r'(\d+\.\d+) out of \d+ stars', tt).group(1) for tt in review_star]

          review_body = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-body"]')]
          # print(f'리뷰갯수: {len(review_title),len(review_date), len(review_star), len(review_body)}')

          print(rep+1, end=" ")
          # review_star = driver.execute_script("return document.querySelector('i[data-hook=\"review-star-rating\"] span.a-icon-alt').textContent")
          # print(f'Title: {review_title[0]}')
          # print(review_date[0])
          # print(review_star[0])
          # print(review_body[0])

          for k in range(len(review_body)):
            review_dict['prd_type'] = temp_text[i]
            review_dict['prd_name'] = prd_name
            review_dict['title'] = review_title[k]
            review_dict['star'] = review_star[k]
            review_dict['review'] = review_body[k]
            review_dict['date'] = review_date[k]
            review_dict['country'] = review_country[k]
            review_all = review_all + [pd.DataFrame([review_dict])]

          driver.find_elements(By.CSS_SELECTOR,'li.a-last')[0].click() # next button

        print('')

        break

      except Exception as e:
        # 해당 제품에서 error 발생시, while문에서 restart
        print(f"Error: {e}")
        print('restart')
        b_stopped = True
        global_i = i
        global_j = j

driver.quit()


SERUM
link 수:10

4=> numbuzin No.4 Collagen 73% Pudding Serum | Wrinkle Care, Aging Dull Skin, Brightening, Elastin, Hyaluronic Acid, Niacinamide | Korean Skin Care for Face, 1.69 fl oz
number of reviews: 99
pages: 1 2 3 4 5 6 7 8 9 10 

5=> numbuzin No.5+ Vitamin Concentrated Serum | Glutathione & Vitamin Serum, Dark Spot, Korean Skincare 1.01fl.oz/30ml
number of reviews: 46
pages: 1 2 3 4 5 

6=> numbuzin No.5 Goodbye Blemish Serum | Vitamin C, Niacinamide, Antioxidant, Brightening Hyperpigmentation, Dark Spots, Acne Scars, Lightweight Gel | Korean Skin Care for Face, 1.69 fl oz
number of reviews: 125
pages: 1 2 3 4 5 6 7 8 9 10 11 12 13 

7=> numbuzin No.6 Deep Sleep Mask Serum | Instant, Long Lasting Hydration, Hyaluronic Acid, Niacinamide, Refreshing Iceland Glacier Water | Korean Skin Care for Face, 1.69 fl oz
number of reviews: 198
pages: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

8=> numbuzin No.7 Mild Green Soothing Serum | Acne-prone Skin, Sensitive Redness, Centel

In [39]:
review_df = pd.concat(review_all).drop_duplicates()
review_df.groupby(['country'])['date'].count()

country
Canada                  4
Germany                 2
India                   1
Japan                 192
Sweden                  1
the United States    1838
Name: date, dtype: int64

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import os, glob

os.listdir('drive/MyDrive/numbuzin')

review_df.to_csv('drive/MyDrive/numbuzin/amazon_numbuzin_review.csv',index=None)

In [51]:
review_df

,prd_type,prd_name,title,star,review,date,country
0,TONER,numbuzin No.1 Pure-Full Calming Herb Toner | A...,Must get!,5.0,This toner gives me the glow on my face! I use...,"December 4, 2023",the United States
0,TONER,numbuzin No.1 Pure-Full Calming Herb Toner | A...,GOOD,4.0,GREAT FOR ACNE PRONE SKIN,"November 25, 2023",the United States
0,TONER,numbuzin No.1 Pure-Full Calming Herb Toner | A...,Great dupe for the anua toner,5.0,I bought this since I thought it’d be a good d...,"October 5, 2023",the United States
0,TONER,numbuzin No.1 Pure-Full Calming Herb Toner | A...,Know how your face reacts to products,4.0,It's good but I think this product was not for...,"June 27, 2023",the United States
0,TONER,numbuzin No.1 Pure-Full Calming Herb Toner | A...,It cleans pores so good!,4.0,I love using this toner because I always see t...,"June 26, 2023",the United States
...,...,...,...,...,...,...,...
0,SHEET MASK,numbuzin No.5 Vitamin Spotlight Sheet Mask,Quick brightening effect,5.0,I was excited to try this Numbuzin No.5 Vitami...,"November 19, 2023",the United States
0,SHEET MASK,numbuzin No.5 Vitamin Spotlight Sheet Mask,"Healthy glow, some frustration",3.0,I take no issue with the performance of these ...,"November 23, 2023",the United States
0,SHEET MASK,numbuzin No.5 Vitamin Spotlight Sheet Mask,Luxury face masks,5.0,These are very good quality face masks that ha...,"November 22, 2023",the United States
0,SHEET MASK,numbuzin No.5 Vitamin Spotlight Sheet Mask,Glowy,5.0,Super glowy and moisturizing! Definitely recom...,"November 29, 2023",the United States
